<a href="https://colab.research.google.com/github/p4zaa/OTTO-Multi-Objective-Recommender-System/blob/main/%5BDEVELOP1_8%5D_OTTO_%E2%80%93_Multi_Objective_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Version Logs
* [View in my Github](https://github.com/p4zaa/OTTO-Multi-Objective-Recommender-System)

# Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc
import pickle

In [2]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install -q torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install -q torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 873.9/873.9 KB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.0/565.0 KB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 KB 27.5 MB/s eta 0:00:00


In [3]:
import pandas as pd, numpy as np
from itertools import product
import io, os, json
import tqdm
from collections import defaultdict

import time

from sklearn.metrics import mean_squared_error

#import plotly.io as pio
#import plotly.express as px
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots
#pio.templates.default = "plotly_white"

import torch
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.datasets import MovieLens
from torch_geometric.nn import to_hetero
from torch_geometric.nn import HeteroConv, GCNConv, SAGEConv, GATConv, Linear, GraphConv
from torch_geometric.data import Data, HeteroData
from torch_geometric.loader import HGTLoader, NeighborLoader, LinkNeighborLoader

!pip install -q sentence-transformers

%matplotlib inline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.7 MB/s eta 0:00:00


In [4]:
!pip3 install -q torch torchvision torchaudio
!pip install -U polars

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 38.3 MB/s eta 0:00:00


In [5]:
import polars as pl

In [6]:
print('Pytorch CUDA Version is ', torch.version.cuda)

Pytorch CUDA Version is  11.6


In [7]:
torch.cuda.empty_cache()

In [8]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Jan  9 13:44:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [9]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [10]:
#!nvidia-smi

In [11]:
torch.cuda.is_available()

True

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Competition Dataset

In [13]:
'''
from google.colab import files
files.upload() # expire any previous token(s) and upload recreated token
!rm -r ~/.kaggle
!mkdir ~/.kaggle
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets list
'''

'\nfrom google.colab import files\nfiles.upload() # expire any previous token(s) and upload recreated token\n!rm -r ~/.kaggle\n!mkdir ~/.kaggle\n!mv ./kaggle.json ~/.kaggle/\n!chmod 600 ~/.kaggle/kaggle.json\n\n!kaggle datasets list\n'

In [14]:
#!kaggle datasets download 'radek1/otto-full-optimized-memory-footprint' -p /content/kaggle/ --unzip

In [15]:
#!kaggle datasets download 'radek1/otto-train-and-test-data-for-local-validation' -p /content/drive/MyDrive/OTTO-Kaggle-Competition/for-local-validation/ --unzip

## Files
- **train.jsonl** - the training data, which contains full session data
  * `session` - the unique session id
  * `events` - the time ordered sequence of events in the session
    * `aid` - the article id (product code) of the associated event
    * `ts` - the Unix timestamp of the event
    * `type` - the event type, i.e., whether a product was clicked, added to the user's cart, or ordered during the session
###### {'clicks': 0, 'carts': 1, 'orders': 2}
- **test.jsonl** - the test data, which contains truncated session data
  * your task is to predict the next `aid` clicked after the session truncation, as well as the the remaining `aids` that are added to `carts` and `orders`; you may predict up to 20 values for each session `type`
- **sample_submission.csv** - a sample submission file in the correct format

In [16]:
# Pandas
#df = pd.read_parquet('/content/kaggle/train.parquet')

# Polars
#df = pl.read_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/dataset/train.parquet')

In [17]:
# sample 10%(frac=0.1) of data
#df = df.sample(frac=0.01, replace=False)

# Polars
#df = df.sample(frac=0.01)

In [18]:
# Save 1% dataset
#df.write_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/train_frac0.01.parquet')

# Load
#df = pl.read_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/train_frac0.01.parquet')

In [19]:
#df = df.to_pandas()

In [20]:
#df[df['aid'] == 1517085]

In [21]:
#df

In [22]:
#df.isna().sum()

In [23]:
#df['type'].unique()

In [24]:
#df.info()

# Construct Graph Data (memory-efficient optimized)

### `edge_index` new code construct

In [25]:
'''
# session index dict
session = df['session'].unique()
source_idx = {id:idx for idx, id in enumerate(session)}

# aid(article id) index dict
aid = df['aid'].unique()
target_idx = {id:idx for idx, id in enumerate(aid)}

# Load data (deserialize)
#with open('/content/drive/MyDrive/OTTO-Kaggle-Competition/source_idx.pkl', 'rb') as file:
#    source_idx = pickle.load(file)

#with open('/content/drive/MyDrive/OTTO-Kaggle-Competition/target_idx.pkl', 'rb') as file:
#    target_idx = pickle.load(file)
'''

"\n# session index dict\nsession = df['session'].unique()\nsource_idx = {id:idx for idx, id in enumerate(session)}\n\n# aid(article id) index dict\naid = df['aid'].unique()\ntarget_idx = {id:idx for idx, id in enumerate(aid)}\n\n# Load data (deserialize)\n#with open('/content/drive/MyDrive/OTTO-Kaggle-Competition/source_idx.pkl', 'rb') as file:\n#    source_idx = pickle.load(file)\n\n#with open('/content/drive/MyDrive/OTTO-Kaggle-Competition/target_idx.pkl', 'rb') as file:\n#    target_idx = pickle.load(file)\n"

In [26]:
'''
connected = df[['session', 'aid']]
connected['session'] = connected['session'].map(source_idx)
connected['aid'] = connected['aid'].map(target_idx)
'''

"\nconnected = df[['session', 'aid']]\nconnected['session'] = connected['session'].map(source_idx)\nconnected['aid'] = connected['aid'].map(target_idx)\n"

In [27]:
'''
source = connected['session']
target = connected['aid']
edge_index = torch.tensor((source.values, target.values))
'''

"\nsource = connected['session']\ntarget = connected['aid']\nedge_index = torch.tensor((source.values, target.values))\n"

In [28]:
#del connected
#gc.collect()

In [29]:
edge_index = torch.load('/content/drive/MyDrive/OTTO-Kaggle-Competition/session-aid_edges_train_and_test.pt')

In [30]:
edge_label = torch.load('/content/drive/MyDrive/OTTO-Kaggle-Competition/edge_label_session-aid_edges_train_and_test.pt')

### Nodes and Edges Attribute

In [31]:
#df['session']

In [32]:
## Nodes Atrributes
session_num_nodes = len(edge_index[0].unique()) #df['session'].nunique()
aid_num_nodes = len(edge_index[1].unique()) #df['aid'].nunique()
#session_id = torch.tensor(list(source_idx.values()), dtype=torch.int64)

aid_features = torch.rand((aid_num_nodes, 32)) # Create (random) article features with shape [num_node_aid, dimensions]
#aid_id = torch.tensor(list(target_idx.values()), dtype=torch.int64)
#aid_features = torch.nn.Embedding(aid_num_nodes, 32)

## Edges Atrributes
edge_index = edge_index
edge_label = edge_label #torch.tensor(df['type'].values).type(torch.int64)

In [33]:
#del df
#gc.collect()

### Construct HeteroData

In [34]:
node_types = {
    'session': {
        'num_nodes': session_num_nodes,
        #'node_id': session_id
    },
    'aid': {
        'x': aid_features,
        #'node_id': aid_id
    }
}

edge_types = {
    ('session', 'event', 'aid'): {
        'edge_index': edge_index,
        'edge_label': edge_label
    }#,
    #('session', 'cart', 'aid'): {
        
    #},
    #('session', 'buy', 'aid'): {
        
    #}
}

In [35]:
data = HeteroData({**node_types, **edge_types})

In [36]:
data

HeteroData(
  session={ num_nodes=14571582 },
  aid={ x=[1855603, 32] },
  (session, event, aid)={
    edge_index=[2, 223644219],
    edge_label=[223644219]
  }
)

In [37]:
data.metadata()

(['session', 'aid'], [('session', 'event', 'aid')])

In [38]:
#del source, target, edge_index, edge_label
#gc.collect()

In [38]:
node_types, edge_types = data.metadata()
print('Node types:', node_types)
print('Edge types:',edge_types)

Node types: ['session', 'aid']
Edge types: [('session', 'event', 'aid')]


In [40]:
#print('Isolated nodes?', data.has_isolated_nodes())
#print('Self loops?', data.has_self_loops())
#print('Undirected graph? ', data.is_undirected())

# Graph-based Modeling [follow [this](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py) sample]

In [41]:
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Construct Undirected Graph

In [42]:
# add sesion features for message passing:
data['session'].x = torch.rand(data['session'].num_nodes, 32)
#data['session'].x = torch.nn.Embedding(data['session'].num_nodes, 32)

In [43]:
# Add user node features for message passing:
#data['session'].x = torch.eye(data['session'].num_nodes, device=device)
#del data['session'].num_nodes

In [44]:
# Add a reverse ('movie', 'rev_rates', 'user') relation for message passing:
data = T.ToUndirected()(data)

In [45]:
del data['aid', 'rev_event', 'session'].edge_label  # Remove "reverse" label.

In [46]:
data

HeteroData(
  session={
    num_nodes=14571582,
    x=[14571582, 32]
  },
  aid={ x=[1855603, 32] },
  (session, event, aid)={
    edge_index=[2, 223644219],
    edge_label=[223644219]
  },
  (aid, rev_event, session)={ edge_index=[2, 223644219] }
)

### [Follow [this sample](https://github.com/pyg-team/pytorch_geometric/blob/master/examples/hetero/hetero_link_pred.py)] Train/Val/Test Link Level Splits

In [47]:
# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.1,
    num_test=0.1,
    neg_sampling_ratio=0.0,
    edge_types=[('session', 'event', 'aid')],
    rev_edge_types=[('aid', 'rev_event', 'session')],
)(data)

### Mini Batch

In [48]:
# Define seed edges:
edge_label_index = train_data['session', 'event', 'aid'].edge_label_index
edge_label = train_data['session', 'event', 'aid'].edge_label

train_loader = LinkNeighborLoader(
    data=train_data,  # TODO
    num_neighbors=[3, 1],  # TODO
    neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=128,
    shuffle=True,
)

# Inspect a sample:
sampled_data = next(iter(train_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

#assert sampled_data["session", "event", "aid"].edge_label_index.size(1) == 128
#assert sampled_data["session", "event", "aid"].edge_label.min() == 0
#assert sampled_data["session", "event", "aid"].edge_label.max() == 2

Sampled mini-batch:
HeteroData(
  session={
    num_nodes=811,
    x=[811, 32]
  },
  aid={ x=[762, 32] },
  (session, event, aid)={
    edge_index=[2, 697],
    edge_label=[128],
    edge_label_index=[2, 128],
    input_id=[128]
  },
  (aid, rev_event, session)={ edge_index=[2, 750] }
)


### [New weight calculation code]

In [49]:
# We have an unbalanced dataset with many labels for rating 3 and 4, and very
# few for 0 and 1. Therefore we use a weighted MSE loss.
weight = torch.bincount(train_data['session', 'aid'].edge_label)
weight = weight.max() / weight

In [50]:
weight

tensor([ 1.0000, 11.5114, 38.9320])

### Model and loss functions

#### Loss function

In [51]:
def weighted_mse_loss(pred, target, weight=None):
    weight = 1. if weight is None else weight[target].to(pred.dtype)
    return (weight * (pred - target.to(pred.dtype)).pow(2)).mean()

#### Model

In [52]:
'''
class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.layers = torch.nn.ModuleList([
            SAGEConv((-1, -1), hidden_channels),
            SAGEConv((-1, -1), out_channels)
        ])

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x, edge_index)
        return x.relu()

class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(2 * hidden_channels, hidden_channels),
            torch.nn.BatchNorm1d(hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, 1)
        )

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)
        return self.layers(z).view(-1)

class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # encoder and decoder
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

        # embedding matrices for sessions and aids
        self.aid_lin = torch.nn.Linear(hidden_channels, hidden_channels)
        self.session_emb = torch.nn.Embedding(data['session'].num_nodes, hidden_channels)
        self.aid_emb = torch.nn.Embedding(data['aid'].num_nodes, hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        #x_dict = {
        #    'session': self.session_emb(data['session'].node_id),
        #    'aid': self.aid_lin(data['aid'].x.float()) + self.aid_emb(data['aid'].node_id)
        #}
        #x_dict = x_dict.copy()
        #x_dict['session'] = self.session_emb.weight
        #x_dict['aid'] = self.aid_emb.weight
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)
'''


"\nclass GNNEncoder(torch.nn.Module):\n    def __init__(self, hidden_channels, out_channels):\n        super().__init__()\n        self.layers = torch.nn.ModuleList([\n            SAGEConv((-1, -1), hidden_channels),\n            SAGEConv((-1, -1), out_channels)\n        ])\n\n    def forward(self, x, edge_index):\n        for layer in self.layers:\n            x = layer(x, edge_index)\n        return x.relu()\n\nclass EdgeDecoder(torch.nn.Module):\n    def __init__(self, hidden_channels):\n        super().__init__()\n        self.layers = torch.nn.Sequential(\n            torch.nn.Linear(2 * hidden_channels, hidden_channels),\n            torch.nn.BatchNorm1d(hidden_channels),\n            torch.nn.ReLU(),\n            torch.nn.Linear(hidden_channels, 1)\n        )\n\n    def forward(self, z_dict, edge_label_index):\n        row, col = edge_label_index\n        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)\n        return self.layers(z).view(-1)\n\nclass Model(to

In [53]:

class GNNEncoder(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = self.conv2(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict['session'][row], z_dict['aid'][col]], dim=-1)

        z = self.lin1(z).relu()
        z = self.lin2(z)

        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        # encoder and decoder
        self.encoder = GNNEncoder(hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr='sum')
        self.decoder = EdgeDecoder(hidden_channels)

        # embedding matrices for sessions and aids
        #self.aid_lin = Linear(hidden_channels, hidden_channels)
        #self.session_emb = torch.nn.Embedding(data['session'].num_nodes, hidden_channels)
        #self.aid_emb = torch.nn.Embedding(data['aid'].num_nodes, hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        #x_dict = {
        #    'session': self.session_emb(data['session'].node_id.to(device)),
        #    'aid': self.aid_lin(data['aid'].x.to(device).float()) + self.aid_emb(data['aid'].node_id.to(device))
        #}
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)

    def get_embedding(self, x_dict, edge_index_dict):
        return self.encoder(x_dict, edge_index_dict)


In [54]:
model = Model(hidden_channels=32).to(device)

In [55]:
# Due to lazy initialization, we need to run one model step so the number
# of parameters can be inferred:

sampled = next(iter(train_loader))

with torch.no_grad():
    model.encoder(sampled_data.to(device).x_dict, sampled_data.to(device).edge_index_dict)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)


In [56]:
del sampled, optimizer
gc.collect()

280

In [57]:
'''
def train():
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data['session', 'aid'].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight)
    loss.backward()
    optimizer.step()
    return float(loss)
'''

"\ndef train():\n    model.train()\n    optimizer.zero_grad()\n    pred = model(train_data.x_dict, train_data.edge_index_dict,\n                 train_data['session', 'aid'].edge_label_index)\n    target = train_data['session', 'aid'].edge_label\n    loss = weighted_mse_loss(pred, target, weight)\n    loss.backward()\n    optimizer.step()\n    return float(loss)\n"

In [58]:
'''
@torch.no_grad()
def test(data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=-1, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse)
'''

"\n@torch.no_grad()\ndef test(data):\n    model.eval()\n    pred = model(data.x_dict, data.edge_index_dict,\n                 data['session', 'aid'].edge_label_index)\n    pred = pred.clamp(min=-1, max=2)\n    target = data['session', 'aid'].edge_label.float()\n    rmse = F.mse_loss(pred, target).sqrt()\n    return float(rmse)\n"

In [59]:

def train(train_data=sampled_data):
    model.train()
    optimizer.zero_grad()
    pred = model(train_data.x_dict, train_data.edge_index_dict,
                 train_data[('session', 'event', 'aid')].edge_label_index)
    target = train_data['session', 'aid'].edge_label
    loss = weighted_mse_loss(pred, target, weight.to(device))
    #loss = F.binary_cross_entropy_with_logits(pred, target)

    # Embedding
    emb_dict = model.get_embedding(train_data.x_dict, train_data.edge_index_dict)

    loss.backward()
    optimizer.step()
    return float(loss), pred, model, emb_dict


In [60]:
'''
def train(model, train_data):
    edge_index_dict, edge_label_index, edge_label = train_data.edge_index_dict, train_data.edge_label_index, train_data.edge_label

    pred = model(edge_index_dict, edge_label_index)
    loss = F.binary_cross_entropy_with_logits(pred, edge_label.to(device).float())
    loss.backward()
    optimizer.step()

    return loss.item(), pred, model
'''

'\ndef train(model, train_data):\n    edge_index_dict, edge_label_index, edge_label = train_data.edge_index_dict, train_data.edge_label_index, train_data.edge_label\n\n    pred = model(edge_index_dict, edge_label_index)\n    loss = F.binary_cross_entropy_with_logits(pred, edge_label.to(device).float())\n    loss.backward()\n    optimizer.step()\n\n    return loss.item(), pred, model\n'

In [61]:
@torch.no_grad()
def test(data=data):
    model.eval()
    #pred = model(data.x_dict, data.edge_index_dict,
    #             data['session', 'aid'].edge_label_index)
    pred = model(data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

In [62]:
@torch.no_grad()
def test(model=model, data=data):
    model.eval()
    pred = model(data.x_dict, data.edge_index_dict,
                 data['session', 'aid'].edge_label_index)
    pred = pred.clamp(min=0, max=2)
    target = data['session', 'aid'].edge_label.float()
    rmse = F.mse_loss(pred, target).sqrt()
    return float(rmse), pred, target

### [mini-batch] Training Heterogenous Link-level GNN

In [63]:
#data = data.to(device)

In [64]:

%%time
import torch.nn.functional as F

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: '{device}'")

#model = Model(hidden_channels=32).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(0, 1):
    total_loss = total_examples = 0
    for sampled_data in tqdm.tqdm(train_loader):
        sampled_data = sampled_data.to(device)
        optimizer.zero_grad()
        loss, pred, model, emb_dict = train(train_data=sampled_data)
        #loss, pred, model = train(model=model, train_data=sampled_data)
        # TODO: Move `sampled_data` to the respective `device`
        # TODO: Run `forward` pass of the model
        # TODO: Apply binary cross entropy via
        # `F.binary_cross_entropy_with_logits(pred, ground_truth)`

        #loss.backward()
        #optimizer.step()
        total_loss += float(loss) * pred.numel()
        total_examples += pred.numel()
    print(f"Epoch: {epoch:03d}, Loss: {total_loss / total_examples:.4f}")


Device: 'cuda'


 16%|█▌        | 221179/1397777 [43:24<3:50:56, 84.91it/s]


KeyboardInterrupt: ignored

### Save and load trained model

In [67]:
path = '/content/drive/MyDrive/OTTO-Kaggle-Competition/model_full_dataset.pt'

In [68]:
# Save the entire model
torch.save(model, path)
model

Model(
  (encoder): GraphModule(
    (conv1): ModuleDict(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
    (conv2): ModuleDict(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
  )
  (decoder): EdgeDecoder(
    (lin1): Linear(64, 32, bias=True)
    (lin2): Linear(32, 1, bias=True)
  )
)

In [17]:
# Load the entire model
model = torch.load(path, map_location=torch.device('cpu'))
model

Model(
  (encoder): GraphModule(
    (conv1): Module(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
    (conv2): Module(
      (session__event__aid): SAGEConv((-1, -1), 32, aggr=mean)
      (aid__rev_event__session): SAGEConv((-1, -1), 32, aggr=mean)
    )
  )
  (decoder): EdgeDecoder(
    (lin1): Linear(64, 32, bias=True)
    (lin2): Linear(32, 1, bias=True)
  )
)

In [68]:
#model.session_emb

In [40]:
model2 = torch.load('/content/drive/MyDrive/OTTO-Kaggle-Competition/model_node2vec.pt')

In [46]:
model3 = torch.load('/content/drive/MyDrive/OTTO-Kaggle-Competition/model_frac0.1.pt')

In [18]:
#data.x_dict

NameError: ignored

In [ ]:
#model.get_embedding(data.to(device).x_dict, data.to(device).edge_index_dict)

In [ ]:
#model.encoder(data.to(device).x_dict, data.to(device).edge_index_dict)

In [ ]:
#trained_model.encoder(data.to(device).x_dict, data.to(device).edge_index_dict)

In [ ]:
#model(val_data.to(device).x_dict,
#      val_data.to(device).edge_index_dict,
#      val_data['session', 'aid'].edge_label_index)

tensor([1.0536, 1.1028, 1.0545,  ..., 1.0439, 1.0466, 1.0864], device='cuda:0',
       grad_fn=<ViewBackward0>)

In [ ]:
#trained_model(val_data.x_dict,
#      val_data.edge_index_dict,
#      val_data['session', 'aid'].edge_label_index)

### Evaluate a Heterogenous Linklevel GNN

In [69]:
# Define the validation seed edges:
edge_label_index = val_data['session', 'event', 'aid'].edge_label_index
edge_label = val_data['session', 'event', 'aid'].edge_label

val_loader = LinkNeighborLoader(
    data=val_data,  # TODO
    num_neighbors=[20, 10],  # TODO
    #neg_sampling_ratio=0.0,  # TODO
    edge_label_index=(('session', 'event', 'aid'), edge_label_index),
    edge_label=edge_label,
    batch_size=3 * 128,
    shuffle=False,
)

sampled_data = next(iter(val_loader))

print("Sampled mini-batch:")
print("===================")
print(sampled_data)

assert sampled_data['session', 'event', 'aid'].edge_label_index.size(1) == 3 * 128
assert sampled_data['session', 'event', 'aid'].edge_label.min() >= 0
assert sampled_data['session', 'event', 'aid'].edge_label.max() <= 2

Sampled mini-batch:
HeteroData(
  session={
    num_nodes=48000,
    x=[48000, 32]
  },
  aid={ x=[40592, 32] },
  (session, event, aid)={
    edge_index=[2, 50972],
    edge_label=[384],
    edge_label_index=[2, 384],
    input_id=[384]
  },
  (aid, rev_event, session)={ edge_index=[2, 68313] }
)


In [72]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import label_binarize

# You need the labels to binarize
labels = [0, 1, 2]

preds = []
ground_truths = []

for sampled_data in tqdm.tqdm(val_loader):
    with torch.no_grad():
        _, pred, ground_truth = test(model=model, data=sampled_data)
        preds.append(pred)
        ground_truths.append(ground_truth)
        # TODO: Collect predictions and ground-truths and write them into
        # `preds` and `ground_truths`.
        #raise NotImplementedError

pred = torch.cat(preds, dim=0).cpu().numpy()
ground_truth = torch.cat(ground_truths, dim=0).cpu().numpy()

# Binarize `pred` with shape (n_samples, n_classes)
pred = label_binarize(pred.astype(int), classes=labels)

# Binarize `ground_truth` with shape (n_samples, n_classes)
ground_truth = label_binarize(ground_truth, classes=labels)

#rmse = _
auc = roc_auc_score(ground_truth, pred, multi_class='ovo', average='weighted')
print()
print(f"Validation AUC: {auc:.4f}")
#print('RMSE: ', rmse)

  0%|          | 113/58241 [00:21<3:07:05,  5.18it/s]


KeyboardInterrupt: ignored

# Evaluation

In [ ]:
#model = train

In [ ]:
#del data
#gc.collect()

In [41]:
model2.state_dict()

OrderedDict([('embedding.weight',
              tensor([[ 0.2494,  0.0784,  0.2418,  ...,  0.1685,  0.0085, -0.1500],
                      [-0.0101,  0.1191,  0.1512,  ..., -0.2910,  0.0166,  0.1297],
                      [-0.3570, -0.1713,  0.1843,  ...,  0.4919, -0.0062, -0.0136],
                      ...,
                      [-0.0022, -0.2023,  0.3768,  ..., -0.2536, -0.0839,  0.3394],
                      [-0.5325, -0.3410,  0.0990,  ..., -0.5023,  0.2621, -0.5723],
                      [-0.2530,  0.0381, -0.0338,  ..., -0.0348,  0.1333, -0.3346]],
                     device='cuda:0'))])

In [43]:
model2.state_dict()['embedding.weight'].shape

torch.Size([1855603, 32])

In [47]:
model3.state_dict()

OrderedDict([('encoder.conv1.session__event__aid.lin_l.weight',
              tensor([[ 0.2087,  0.1432,  0.1538,  ..., -0.1009, -0.1565, -0.0229],
                      [ 0.0291,  0.0585,  0.0842,  ..., -0.0779,  0.2441,  0.2059],
                      [-0.0990, -0.2112,  0.1550,  ...,  0.1726,  0.0990, -0.0743],
                      ...,
                      [-0.6443, -0.1546,  0.7859,  ...,  0.2159, -0.2733,  0.4864],
                      [ 0.0192, -0.1633, -0.1616,  ..., -0.0184, -0.0451,  0.0491],
                      [ 0.0087, -0.0999, -0.0022,  ...,  0.0833, -0.0433,  0.1291]],
                     device='cuda:0')),
             ('encoder.conv1.session__event__aid.lin_l.bias',
              tensor([-0.1622,  0.0261, -0.1761, -0.3527, -0.2081,  0.1091, -0.1533, -0.1407,
                      -0.0246, -0.2430, -0.3877, -0.3214, -0.2803, -0.0109, -0.0713, -0.0145,
                      -0.2210, -0.3043, -0.2705, -0.0762, -0.3324, -0.0199, -0.1911, -0.2275,
                    

In [49]:
model3.state_dict()['encoder.conv2.session__event__aid.lin_l.weight'].shape

torch.Size([32, 32])

In [69]:
model.state_dict()

OrderedDict([('encoder.conv1.session__event__aid.lin_l.weight',
              tensor([[-0.1586,  0.0805, -0.1247,  ...,  0.1056,  0.0450,  0.0194],
                      [-0.2260,  0.0898, -0.0476,  ..., -0.0876,  0.0161, -0.0265],
                      [-0.0876, -0.1565,  0.1321,  ...,  0.0678, -0.0924,  0.0265],
                      ...,
                      [-0.0777,  0.0588, -0.0452,  ..., -0.1548,  0.0844, -0.1329],
                      [ 0.0378, -0.1818,  0.0156,  ...,  0.0212,  0.1374, -0.1866],
                      [-0.0321, -0.1472,  0.1248,  ..., -0.0274,  0.0790, -0.1516]],
                     device='cuda:0')),
             ('encoder.conv1.session__event__aid.lin_l.bias',
              tensor([-0.1938, -0.1329, -0.1203, -0.1646,  0.0737, -0.1332,  0.0782,  0.1005,
                      -0.1241, -0.0613, -0.3342, -0.0192, -0.0933, -0.1976,  0.0037, -0.0312,
                      -0.1802, -0.0310, -0.0718,  0.0525,  0.0502,  0.1002, -0.1570, -0.0791,
                    

In [70]:
model.state_dict()['encoder.conv1.aid__rev_event__session.lin_l.weight'].shape

torch.Size([32, 32])

In [50]:
def get_edge_index(df):
    return torch.tensor(np.transpose(df[['session', 'aid']].to_numpy()), dtype=torch.long)

In [20]:
!pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.0/648.0 KB 39.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.1-cp38-cp38-linux_x86_64.whl size=394073 sha256=1b233cacfd31d0e2f004b40e7596ef65b8fcd35445a1a6505de99f8cca8a95af
  Stored in directory: /root/.cache/pip/wheels/f9/93/19/30511c4a9ae6b4937455a134c34a39e13943e2c6f46fcd2ed2
Successfully built annoy


In [28]:
%%time
from annoy import AnnoyIndex

index = AnnoyIndex(32, metric='angular')

for idx, idx_embedding in enumerate(model.state_dict()['encoder.conv2.session__event__aid.lin_r.weight'].cpu()):
  index.add_item(idx, idx_embedding)

index.build(10)

CPU times: user 5.89 ms, sys: 0 ns, total: 5.89 ms
Wall time: 5.22 ms


True

In [ ]:
#del model, loader, optimizer
#gc.collect()
#torch.cuda.empty_cache()

### Validation/Inference

In [ ]:
# source_idx -> dict{id:idx}
# target_idx -> dict{id:idx}

In [24]:
def evaluate(path,mode="validation",n_neighbors=20):


    test = pl.read_parquet(path)

    session_types = ['clicks', 'carts', 'orders']

    test_session_AIDs = test.to_pandas().reset_index(drop=True).groupby('session')['aid'].apply(list)
    '''
    session
    12899779                                              [59625]
    12899780           [1142000, 582732, 973453, 736515, 1142000]
    12899781    [141736, 199008, 57315, 194067, 199008, 199008...
    12899782    [1669402, 1494780, 1494780, 1494780, 1494780, ...
    12899783    [255297, 1114789, 255297, 300127, 198385, 3001...
                                      ...                        
    14571577                                            [1141710]
    14571578                                             [519105]
    14571579                                             [739876]
    14571580                                             [202353]
    14571581                                            [1100210]
    Name: aid, Length: 1671803, dtype: object
    '''

    test_session_types = test.to_pandas().reset_index(drop=True).groupby('session')['type'].apply(list)
    '''
    session
    12899779                                                  [0]
    12899780                                      [0, 0, 0, 0, 0]
    12899781                    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]
    12899782    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...
    12899783                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                                      ...                        
    14571577                                                  [0]
    14571578                                                  [0]
    14571579                                                  [0]
    14571580                                                  [0]
    14571581                                                  [0]
    Name: type, Length: 1671803, dtype: object
    '''

    del test
    gc.collect()

    labels = []

    type_weight_multipliers = {0: 1, 1: 6, 2: 3}

    for AIDs, types in zip(test_session_AIDs, test_session_types):
        # AIDs: list of connected aids in each session
        # types: list of connected types in each session
        if len(AIDs) >= 20: # this session have at least 20 aids connected
            # if we have enough aids (over equals 20) we don't need to look for candidates! we just use the old logic
            weights = np.logspace(0.1,1,len(AIDs), base=2, endpoint=True) - 1
            aids_temp = defaultdict(lambda: 0)
            for aid, w, t in zip(AIDs, weights, types): 
                # {aid : weight}
                aids_temp[aid] += w * type_weight_multipliers[t]

            # list of sorted dictionary key by value
            sorted_aids = [k for k, v in sorted(aids_temp.items(), key=lambda item: -item[1])]
            labels.append(sorted_aids[:20]) # only top 20
        else:
            # here we don't have 20 aids to output -- we will use word2vec embeddings to generate candidates!
            AIDs = list(dict.fromkeys(AIDs[::-1]))

            # let's grab the most recent aid
            most_recent_aid = AIDs[0]

            # and look for some neighbors!
            nns = [i for i in index.get_nns_by_item(most_recent_aid, n_neighbors+1)[1:]]


            labels.append((AIDs+nns)[:n_neighbors])

    labels_as_strings = [' '.join([str(l) for l in lls]) for lls in labels]

    predictions = pd.DataFrame(data={'session_type': test_session_AIDs.index, 'labels': labels_as_strings})

    prediction_dfs = []

    for st in session_types:
        modified_predictions = predictions.copy()
        modified_predictions.session_type = modified_predictions.session_type.astype('str') + f'_{st}'
        prediction_dfs.append(modified_predictions)

    sub = pd.concat(prediction_dfs).reset_index(drop=True)
    
    del prediction_dfs, predictions,labels_as_strings, labels, test_session_types,test_session_AIDs
    gc.collect()
    if mode=="test":
        sub.to_csv("submission.csv",index=False)
        return sub
    else:

        sub['labels_2'] = sub['labels'].apply(lambda x : [int(s) for s in x.split(' ')])
        submission = pd.DataFrame()
        submission['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
        submission['type'] = sub.session_type.apply(lambda x: x.split('_')[1])
        submission['labels'] = sub.labels_2.apply(lambda x : [item for item in x[:] ]) #.apply(lambda x: [int(i) for i in x.split(',')[:20]])
        test_labels = pd.read_parquet('/content/drive/MyDrive/OTTO-Kaggle-Competition/for-local-validation/test_labels.parquet')
        test_labels = test_labels.merge(submission, how='left', on=['session', 'type'])
        del sub,submission
        gc.collect()
        gc.collect()
        test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
        recall_per_type = test_labels.groupby(['type'])['hits'].sum() / test_labels.groupby(['type'])['gt_count'].sum() 
        score = (recall_per_type * pd.Series({'clicks': 0.1, 'carts': 0.30, 'orders': 0.60})).sum()

        return score

### Validation

In [25]:
path = "/content/drive/MyDrive/OTTO-Kaggle-Competition/for-local-validation/test.parquet"
validation_score = evaluate(path,mode="validation",n_neighbors=20)

IndexError: ignored

In [26]:
%debug

> <ipython-input-24-95bee18874b9>(71)evaluate()
     69 
     70             # and look for some neighbors!
---> 71             nns = [i for i in index.get_nns_by_item(most_recent_aid, n_neighbors+1)[1:]]
     72 
     73 

ipdb> nns
*** NameError: name 'nns' is not defined
ipdb> index
ipdb> most_recent_aid
11830
ipdb> index.get_nns_by_item(11830, 21)
*** IndexError: Item index larger than the largest item index
ipdb> index.size
*** AttributeError: 'annoy.Annoy' object has no attribute 'size'
ipdb> index.shape
*** AttributeError: 'annoy.Annoy' object has no attribute 'shape'
ipdb> df
*** NameError: name 'df' is not defined
ipdb> quit



PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/lib/python3.8/bdb.py", line 359, in set_quit
    sys.settrace(None)



In [ ]:
print(validation_score)

NameError: ignored

# Submission

Submission File
For each `session` id and `type` combination in the test set, you must predict the `aid` values in the `label` column, which is space delimited. You can predict up to 20 `aid` values per row. The file should contain a header and have the following format:

```
session_type,labels
12906577_clicks,135193 129431 119318 ...
12906577_carts,135193 129431 119318 ...
12906577_orders,135193 129431 119318 ...
12906578_clicks, 135193 129431 119318 ...
etc.
```

In [ ]:
path = "/content/drive/MyDrive/OTTO-Kaggle-Competition/dataset/test.parquet"
test_submission = evaluate(path,mode="test",n_neighbors=20)

In [ ]:
test_submission